### Extracting features of 2020 movies from Wikipedia

In [115]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [116]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [117]:
df1=pd.read_html(url,header=0)[3]
df2=pd.read_html(url,header=0)[4]
#df3=pd.read_html(url,header=0)[5]
#df4=pd.read_html(url,header=0)[6]

In [118]:
df = df1.append(df2,ignore_index=True)

In [119]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
87,JUNE,19,7500,Amazon Studios,Patrick Vollrath (director/screenplay); Joseph...,[88]
88,JUNE,24,Athlete A,Netflix,"Bonni Cohen, Jon Shenk (director)",[89]
89,JUNE,26,Eurovision Song Contest: The Story of Fire Saga,Netflix / Gary Sanchez Productions / Gloria Sa...,"David Dobkin (director); Will Ferrell, Andrew ...",[90]
90,JUNE,26,Irresistible,Focus Features / Plan B Entertainment,Jon Stewart (director/screenplay); Steve Carel...,[91]


In [120]:
df_2020 = df[['Title','Cast and crew']]

In [121]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
87,7500,Patrick Vollrath (director/screenplay); Joseph...
88,Athlete A,"Bonni Cohen, Jon Shenk (director)"
89,Eurovision Song Contest: The Story of Fire Saga,"David Dobkin (director); Will Ferrell, Andrew ..."
90,Irresistible,Jon Stewart (director/screenplay); Steve Carel...


In [122]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '535a117741e6494651ee0dd1da293dc4'

In [123]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [124]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-124-fdc91464740e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


In [126]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery
...,...,...,...
87,7500,Patrick Vollrath (director/screenplay); Joseph...,Action Drama Thriller
88,Athlete A,"Bonni Cohen, Jon Shenk (director)",Documentary
89,Eurovision Song Contest: The Story of Fire Saga,"David Dobkin (director); Will Ferrell, Andrew ...",Comedy Music
90,Irresistible,Jon Stewart (director/screenplay); Steve Carel...,Comedy Drama


In [127]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [128]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-128-bc52fe9b0c35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [129]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [130]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-130-42c5f763f942>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [131]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [132]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [133]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [134]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [135]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...,...
87,7500,Patrick Vollrath (director/screenplay); Joseph...,Action Drama Thriller,Patrick Vollrath,Joseph Gordon-Levitt,NaN,NaN
88,Athlete A,"Bonni Cohen, Jon Shenk (director)",Documentary,"Bonni Cohen, Jon Shenk",Bonni Cohen,Jon Shenk (director),NaN
89,Eurovision Song Contest: The Story of Fire Saga,"David Dobkin (director); Will Ferrell, Andrew ...",Comedy Music,David Dobkin,Will Ferrell,Rachel McAdams,Dan Stevens
90,Irresistible,Jon Stewart (director/screenplay); Steve Carel...,Comedy Drama,Jon Stewart,Steve Carell,Chris Cooper,Mackenzie Davis


In [136]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [137]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [138]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,Inherit the Viper
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,The Sonata
...,...,...,...,...,...,...
87,Patrick Vollrath,Joseph Gordon-Levitt,NaN,NaN,Action Drama Thriller,7500
88,"Bonni Cohen, Jon Shenk",Bonni Cohen,Jon Shenk (director),NaN,Documentary,Athlete A
89,David Dobkin,Will Ferrell,Rachel McAdams,Dan Stevens,Comedy Music,Eurovision Song Contest: The Story of Fire Saga
90,Jon Stewart,Steve Carell,Chris Cooper,Mackenzie Davis,Comedy Drama,Irresistible


In [139]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [140]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     1
actor_3_name     8
genres           0
movie_title      0
comb             8
dtype: int64

In [141]:
new_df20 = new_df20.dropna(how='any')

In [142]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [143]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

<ipython-input-143-83e83e91073b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [144]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,the sonata,Freya Tingley Simon Abkarian Rutger Hauer Andr...
...,...,...,...,...,...,...,...
85,Olivier Assayas,Penélope Cruz,Édgar Ramírez,Gael García Bernal,Drama History Thriller,wasp network,Penélope Cruz Édgar Ramírez Gael García Bernal...
86,Elissa Down,Elissa Down (director); Sofia Carson,Enrico Colantoni,Wolfgang Novogratz,Comedy Music Drama,feel the beat,Elissa Down (director); Sofia Carson Enrico Co...
89,David Dobkin,Will Ferrell,Rachel McAdams,Dan Stevens,Comedy Music,eurovision song contest: the story of fire saga,Will Ferrell Rachel McAdams Dan Stevens David ...
90,Jon Stewart,Steve Carell,Chris Cooper,Mackenzie Davis,Comedy Drama,irresistible,Steve Carell Chris Cooper Mackenzie Davis Jon ...


In [145]:
old_df = pd.read_csv('../datasets/data_2019.csv')

FileNotFoundError: [Errno 2] File ../datasets/data_2019.csv does not exist: '../datasets/data_2019.csv'

In [146]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [147]:
final_df = old_df.append(new_df20,ignore_index=True)

In [148]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5948,Olivier Assayas,Penélope Cruz,Édgar Ramírez,Gael García Bernal,Drama History Thriller,wasp network,Penélope Cruz Édgar Ramírez Gael García Bernal...
5949,Elissa Down,Elissa Down (director); Sofia Carson,Enrico Colantoni,Wolfgang Novogratz,Comedy Music Drama,feel the beat,Elissa Down (director); Sofia Carson Enrico Co...
5950,David Dobkin,Will Ferrell,Rachel McAdams,Dan Stevens,Comedy Music,eurovision song contest: the story of fire saga,Will Ferrell Rachel McAdams Dan Stevens David ...
5951,Jon Stewart,Steve Carell,Chris Cooper,Mackenzie Davis,Comedy Drama,irresistible,Steve Carell Chris Cooper Mackenzie Davis Jon ...


In [149]:
final_df.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)

In [150]:
final_df.to_csv('../datasets/main_data.csv',index=False)

In [151]:
final_df.count()

director_name    5926
actor_1_name     5926
actor_2_name     5926
actor_3_name     5926
genres           5926
movie_title      5926
comb             5926
dtype: int64

In [152]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64